# Example creating custom models with GPT-2




In [1]:
# Import various things
import neural_nlp
from neural_nlp import score
from neural_nlp.models import model_pool, model_layers, implementations
from brainscore.utils import LazyLoad
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config

/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
def gpt2_test():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained(
        "gpt2", output_hidden_states=True
    )
    wrapper = implementations._PytorchTransformerWrapper(
        identifier="gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12))),
        sentence_average=implementations.word_last
    )
    return wrapper

In [3]:
model_pool["gpt2-test"] = LazyLoad(lambda: gpt2_test())
model_layers["gpt2-test"] = list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12)))
assert model_layers["gpt2-test"] == model_layers["gpt2"]


In [4]:
print(GPT2Tokenizer.from_pretrained("gpt2"))

In [5]:
# Run and evaluate the model generated in gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="gpt2-test")

<xarray.Score (layer: 13, aggregation: 2)>
array([[0.04500979, 0.10728886],
       [0.60563093, 0.13442728],
       [0.67513002, 0.02300425],
       [0.63005166, 0.07155539],
       [0.5024158 , 0.08306366],
       [0.48248263, 0.12891596],
       [0.50950414, 0.09476689],
       [0.559599  , 0.10482938],
       [0.63031937, 0.18250217],
       [0.72617442, 0.14373017],
       [0.80979421, 0.12343126],
       [0.88677038, 0.05182417],
       [0.9121889 , 0.11140106]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.11'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.01433...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        gpt2-test
    benchmark:    Pereira2018-encoding

In [6]:
# default_model = GPT2Mode.from_pretrained("gpt2", output_hidden_states=True)
# default_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# default_identifier = "gpt2"
# def model_test(identifier="gpt2-test", model=default_model, tokenizer=default_tokenizer):
#     implementations

def untrained_gpt2_test():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    config = GPT2Config.from_pretrained("gpt2")
    
    # Schrimpf et al. initialize the model this way
    # See lines approximately 1325-1330 in implementations.py
    model = GPT2Model(config)
    state_dict = model.state_dict()  # Capture the initial random weights
    
    # Force the initial random weights back onto the pre-trained model
    model = GPT2Model.from_pretrained("gpt2", output_hidden_states=True, state_dict=state_dict)
    model = model.float()
    
    wrapper = implementations._PytorchTransformerWrapper(
        identifier="untrained-gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12))),
        sentence_average=implementations.word_last
    )
    return wrapper

In [7]:
model_pool["untrained-gpt2-test"] = LazyLoad(lambda: untrained_gpt2_test())
model_layers["untrained-gpt2-test"] = list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12)))

In [8]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-gpt2-test")

<xarray.Score (layer: 13, aggregation: 2)>
array([[0.04223047, 0.03959577],
       [0.81282733, 0.06704036],
       [0.86791751, 0.08255504],
       [0.87549202, 0.10991611],
       [0.86945091, 0.08111384],
       [0.88217028, 0.10755938],
       [0.88257703, 0.10618529],
       [0.88984818, 0.13134028],
       [0.88688   , 0.12774002],
       [0.88666312, 0.10500984],
       [0.87644635, 0.08464181],
       [0.8730716 , 0.07575206],
       [0.85522824, 0.06460767]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.11'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.01345...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-gpt2-test
    benchmark:    Pereira2018-encoding

In [9]:
# Use different embeddings (dimensions) for gpt-2 layers. Consistently use 24 layers
n_layer = 24
def untrained_n_embedding_gpt2_test(embeddings):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    config = GPT2Config.from_pretrained("gpt2", output_hidden_states=True)
    config.n_layer = n_layer
    config.n_embed = embeddings
    model = GPT2Model(config)

    wrapper = implementations._PytorchTransformerWrapper(
        identifier=f"untrained-{embeddings}-embedding-gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(n_layer))),
        sentence_average=implementations.word_last
    )
    return wrapper


In [10]:
# Set up several untrained models of different number of embeddings
for embeddings in [10, 20, 50, 100, 500, 768, 1000]:
    
    model_pool[f"untrained-{embeddings}-embedding-gpt2-test"] = (
        lambda e: LazyLoad(
            lambda: untrained_n_embedding_gpt2_test(e)
        )
    )(embeddings)
    model_layers[f"untrained-{embeddings}-embedding-gpt2-test"] = list(
        ("drop",) + tuple(f"encoder.h.{i}" for i in range(n_layer))
    )

In [11]:
print(model_pool.keys())

dict_keys(['sentence-length', 'word-position', 'random-embedding', 'skip-thoughts', 'skip-thoughts-untrained', 'lm_1b', 'lm_1b-untrained', 'word2vec', 'word2vec-untrained', 'glove', 'glove-untrained', 'transformer', 'transformer-untrained', 'ETM', 'ETM-untrained', 'bert-base-uncased', 'bert-base-multilingual-cased', 'bert-large-uncased', 'bert-large-uncased-whole-word-masking', 'openaigpt', 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'transfo-xl-wt103', 'xlnet-base-cased', 'xlnet-large-cased', 'xlm-mlm-en-2048', 'xlm-mlm-enfr-1024', 'xlm-mlm-xnli15-1024', 'xlm-clm-enfr-1024', 'xlm-mlm-100-1280', 'roberta-base', 'roberta-large', 'distilroberta-base', 'distilbert-base-uncased', 'ctrl', 'albert-base-v1', 'albert-base-v2', 'albert-large-v1', 'albert-large-v2', 'albert-xlarge-v1', 'albert-xlarge-v2', 'albert-xxlarge-v1', 'albert-xxlarge-v2', 't5-small', 't5-base', 't5-large', 't5-3b', 't5-11b', 'xlm-roberta-base', 'xlm-roberta-large', 'bert-base-uncased-untrained', 'bert-b

In [12]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-10-embedding-gpt2-test")

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.03872582, 0.11402085],
       [0.78971892, 0.09692257],
       [0.8443061 , 0.05445144],
       [0.87057063, 0.11833863],
       [0.88990241, 0.05671713],
       [0.90485135, 0.10153557],
       [0.89898959, 0.05296617],
       [0.88986379, 0.07374882],
       [0.88922881, 0.10531158],
       [0.91897324, 0.14983308],
       [0.90957477, 0.15278893],
       [0.88874723, 0.13065168],
       [0.89102288, 0.13137768],
       [0.90803605, 0.11847565],
       [0.90466126, 0.12600947],
       [0.89139833, 0.11648843],
       [0.89153821, 0.11722361],
       [0.89817201, 0.13777728],
       [0.89971122, 0.13742477],
       [0.9069629 , 0.13471655],
       [0.90708145, 0.1287673 ],
       [0.93579999, 0.13363783],
       [0.9296681 , 0.11621894],
       [0.92700357, 0.1387064 ],
       [0.93294663, 0.13229929]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.01233...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-10-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [13]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-20-embedding-gpt2-test")

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.01443505, 0.07044489],
       [0.74201145, 0.06103926],
       [0.82170769, 0.20037037],
       [0.84818345, 0.0803312 ],
       [0.86078558, 0.078837  ],
       [0.89149817, 0.13604651],
       [0.87487307, 0.10822428],
       [0.90259087, 0.10967738],
       [0.87831357, 0.15388197],
       [0.9023393 , 0.14356523],
       [0.91863991, 0.12113445],
       [0.91530603, 0.11822879],
       [0.91036752, 0.11607576],
       [0.9066447 , 0.13300864],
       [0.90929944, 0.12292178],
       [0.91481066, 0.14881989],
       [0.92369131, 0.13603618],
       [0.91032872, 0.14612877],
       [0.89732672, 0.13395828],
       [0.8956142 , 0.12990409],
       [0.89085038, 0.12291545],
       [0.8944899 , 0.12157745],
       [0.90079962, 0.13063956],
       [0.89262368, 0.13887001],
       [0.89012281, 0.13025573]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00459...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-20-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [14]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-50-embedding-gpt2-test")

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.00283364, 0.03440709],
       [0.77787181, 0.0696091 ],
       [0.84637071, 0.05412396],
       [0.86249201, 0.10005908],
       [0.85287524, 0.06516104],
       [0.88167335, 0.09037197],
       [0.8868236 , 0.13516442],
       [0.84660862, 0.13251538],
       [0.86125045, 0.10251875],
       [0.87037009, 0.1149067 ],
       [0.87375385, 0.13195403],
       [0.88536236, 0.1393623 ],
       [0.91314191, 0.13739212],
       [0.91910564, 0.14139671],
       [0.92833929, 0.13915581],
       [0.93618848, 0.10608457],
       [0.93124467, 0.09316814],
       [0.94104967, 0.10320042],
       [0.93200857, 0.1305396 ],
       [0.91942813, 0.1040272 ],
       [0.92482263, 0.06777612],
       [0.90941531, 0.0987366 ],
       [0.90757341, 0.09042905],
       [0.90556225, 0.08243331],
       [0.92334224, 0.07345216]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00090...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-50-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [15]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-100-embedding-gpt2-test")

<xarray.Score (layer: 25, aggregation: 2)>
array([[-0.00237707,  0.06207606],
       [ 0.80784967,  0.07298576],
       [ 0.8568618 ,  0.13641384],
       [ 0.86069286,  0.16049764],
       [ 0.88624691,  0.14548278],
       [ 0.89373423,  0.13948103],
       [ 0.89521737,  0.14038527],
       [ 0.88493891,  0.15713547],
       [ 0.88044761,  0.15831589],
       [ 0.88935238,  0.19472925],
       [ 0.89724713,  0.16704635],
       [ 0.90224772,  0.14370174],
       [ 0.91100208,  0.13880974],
       [ 0.91271786,  0.14585289],
       [ 0.90579359,  0.13951717],
       [ 0.90321663,  0.11289885],
       [ 0.90974557,  0.08625501],
       [ 0.91637223,  0.08904756],
       [ 0.92875084,  0.11165338],
       [ 0.93812923,  0.10904582],
       [ 0.92809192,  0.10555831],
       [ 0.92753711,  0.11030191],
       [ 0.92975045,  0.11877295],
       [ 0.92073381,  0.10979289],
       [ 0.9275986 ,  0.08192335]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[-0.0007...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-100-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [16]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-500-embedding-gpt2-test")

layers:   0%|                                            | 0/25 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 66/66 [00:00<00:00, 89.71it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 270.03it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 72/72 [00:00<00:00, 104.76it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 269.00it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 107.01it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 263.42it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 90.94it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.09it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 269.44it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 67/67 [00:00<00:00, 88.79it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.35it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 103.79it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.17it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 61/61 [00:00<00:00, 89.75it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.13it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00<00:00, 83.75it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.55it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 83.46it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 275.66it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00<00:00, 89.25it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.64it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 96.81it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.89it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 95.37it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.90it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00<00:00, 87.51it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.33it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 82.91it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.35it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 83.51it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.29it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<00:00, 92.48it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.50it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 94.35it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 275.60it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00<00:00, 93.60it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.64it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 89.96it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.54it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 84.89it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.24it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 92.46it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.99it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 37/37 [00:00<00:00, 88.10it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 269.65it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 93.15it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.11it/s]
Using cls_token, but 

...



cross-validation:  80%|█████████████████████▌     | 4/5 [02:19<00:34, 34.85s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [02:53<00:00, 34.79s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:28<12:12, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:28<12:12, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:04<00:17,  4.



cross-validation:  80%|█████████████████████▌     | 4/5 [00:41<00:10, 10.39s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [00:52<00:00, 10.40s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:52<07:50, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:52<07:50, 52.28

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:08<00:32,  8.11s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:16<00:24,  8.13s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:24<00:16,  8.16s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:32<00:08,  8.24s/it]

cro

cross-validation:  20%|█████▍                     | 1/5 [00:02<00:11,  2.78s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:05<00:08,  2.78s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:08<00:05,  2.79s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:11<00:02,  2.79s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.79s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:42<00:52, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:42<00:52, 52.53s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:14<00:57, 14.50s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:29<00:44, 14.73s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:44<00:29, 14.79s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:58<00:14, 14.77s/it]

cross-validation:

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:23<01:35, 23.95s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:48<01:12, 24.11s/it]

cross-validation:  60%|████████████████▏          | 3/5 [01:12<00:48, 24.27s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [01:36<00:24, 24.31s/it]

cross-validation: 100%|███████████████████████████| 5/5 [02:01<00:00, 24.28s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:56<07:32, 90.56s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:56<07:32, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:06<00:26,  6.53s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:13<00:19,  6.55s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:19<00:13,  6.50s/it]

cross-validation:

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.00797014, 0.10464605],
       [0.76781379, 0.04342871],
       [0.82948868, 0.05887671],
       [0.85295537, 0.06961845],
       [0.87000077, 0.07907999],
       [0.86200738, 0.08202215],
       [0.84464563, 0.0813002 ],
       [0.86919567, 0.0913983 ],
       [0.87032153, 0.06888503],
       [0.86434145, 0.04658686],
       [0.86632107, 0.04704417],
       [0.86830668, 0.01354071],
       [0.87835147, 0.00839595],
       [0.87068698, 0.04241107],
       [0.88660105, 0.0518952 ],
       [0.87269697, 0.03771935],
       [0.87214102, 0.05060264],
       [0.87251597, 0.04051833],
       [0.87109708, 0.01908866],
       [0.87237254, 0.02879325],
       [0.8793684 , 0.06724307],
       [0.88540519, 0.09083573],
       [0.88453606, 0.07067266],
       [0.88385174, 0.08259275],
       [0.88017375, 0.07821346]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00253...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-500-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [17]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-768-embedding-gpt2-test")

layers:   0%|                                            | 0/25 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 66/66 [00:00<00:00, 91.30it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.56it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 72/72 [00:00<00:00, 105.03it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.19it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 102.95it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 279.77it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 90.58it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.16it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 90.83it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.98it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 67/67 [00:00<00:00, 90.50it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.76it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 104.72it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.45it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 61/61 [00:00<00:00, 89.45it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.79it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00<00:00, 83.85it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 269.39it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 84.96it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.26it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00<00:00, 89.17it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.74it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 93.32it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.64it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 94.39it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 276.45it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00<00:00, 86.98it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.04it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 80.28it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.90it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 81.59it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.09it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<00:00, 92.19it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.40it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 94.43it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.37it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00<00:00, 91.79it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.04it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 91.17it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.20it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 83.90it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 287.84it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 92.97it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.93it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 37/37 [00:00<00:00, 87.40it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.76it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 91.13it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.26it/s]
Using cls_token, but 

...



cross-validation: 100%|███████████████████████████| 5/5 [02:50<00:00, 34.06s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:23<11:58, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:23<11:58, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:04<00:16,  4.21s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|██████████▊                | 2/5 [00:08<00:12,  4.



cross-validation: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.16s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:51<07:39, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:51<07:39, 51.05

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:07<00:31,  7.98s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:15<00:23,  7.95s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:23<00:15,  7.99s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:31<00:08,  8.01s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:40<00:00,  8.03s/it]

{'experiment': '384sentences', 'atlas': 'DMN'}:  20%|▏| 2/10 [01:31<05:58, 44.78
{'experiment': '384sentences', 'atlas': 'visual'}:  20%|▏| 2/10 [01:31<05:58, 44

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documen

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.75s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:43<00:52, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:43<00:52, 52.27s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:14<00:57, 14.27s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:28<00:42, 14.32s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:42<00:28, 14.29s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:57<00:14, 14.23s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:11<00:00, 14.26s/it]

{'experiment': '243sentences', 'atlas': 'MD'}: 100%|█| 10/10 [10:55<00:00, 65.53
cartesian product:   0%|                                 | 0/10 [00:00<?, ?it/s]
{'experiment': '384sentences', 'atlas': 'language'}:   0%| | 0/10 [00:00<?, ?it/

cross-validation:  

cross-validation:  80%|█████████████████████▌     | 4/5 [01:34<00:23, 23.52s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:58<00:00, 23.66s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:50<07:24, 88.91s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:50<07:24, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:06<00:25,  6.39s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:12<00:19,  6.39s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:19<00:12,  6.39s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:25<00:06,  6.46s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:32<00:00,  6.44s/it]

{'experiment': '243sentences', 'atlas': 'language'}:  60%|▌| 6/10 [07:22<04:38, 
{'experiment': '243sentences', 'atlas': 'DMN'}:  60%|▌| 6/10 [07:22<04:38, 69.70

cross-validation: 

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.0238217 , 0.03619694],
       [0.8350281 , 0.11876474],
       [0.83760476, 0.09858681],
       [0.82974924, 0.09156054],
       [0.8688184 , 0.13441315],
       [0.87751793, 0.11235777],
       [0.89474777, 0.09841045],
       [0.89002765, 0.12487757],
       [0.87831834, 0.12138212],
       [0.87177531, 0.11744527],
       [0.8669203 , 0.14306536],
       [0.85192657, 0.11176438],
       [0.85390071, 0.1167996 ],
       [0.84555342, 0.12766524],
       [0.84857026, 0.1322117 ],
       [0.85698976, 0.13120185],
       [0.85895328, 0.10980897],
       [0.87226678, 0.13873197],
       [0.86821925, 0.12885788],
       [0.85925372, 0.1369715 ],
       [0.84914109, 0.14244658],
       [0.86090512, 0.12897883],
       [0.86298867, 0.12698667],
       [0.87679832, 0.16543476],
       [0.87568041, 0.15414572]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00758...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-768-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [18]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 4 hours)

score(benchmark="Pereira2018-encoding", model="untrained-1000-embedding-gpt2-test")

cartesian product:   0%|                                 | 0/10 [00:00<?, ?it/s]
{'experiment': '384sentences', 'atlas': 'language'}:   0%| | 0/10 [00:00<?, ?it/

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:09<00:39,  9.94s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|██████████▊                | 2/5 [00:19<00:29,  9.93s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packa

{'experiment': '243sentences', 'atlas': 'language'}:  60%|▌| 6/10 [06:55<04:23, 
{'experiment': '243sentences', 'atlas': 'DMN'}:  60%|▌| 6/10 [06:55<04:23, 65.86

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:05<00:21,  5.32s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:10<00:15,  5.31s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:15<00:10,  5.27s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:20<00:05,  5.22s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:26<00:00,  5.24s/it]

{'experiment': '243sentences', 'atlas': 'DMN'}:  70%|▋| 7/10 [07:22<02:38, 52.97
{'experiment': '243sentences', 'atlas': 'visual'}:  70%|▋| 7/10 [07:22<02:38, 52

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:19<01:16, 19.15s/it]

cross-validation: 

cross-validation:  20%|█████▍                     | 1/5 [00:04<00:17,  4.26s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|██████████▊                | 2/5 [00:08<00:12,  4.25s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  60%|████████████████▏          | 3/5 [00:12<00:08,  4.27s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConsta

cross-validation:  40%|██████████▊                | 2/5 [00:16<00:24,  8.20s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:24<00:16,  8.21s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:32<00:08,  8.18s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:40<00:00,  8.19s/it]

{'experiment': '384sentences', 'atlas': 'DMN'}:  20%|▏| 2/10 [01:33<06:06, 45.77
{'experiment': '384sentences', 'atlas': 'visual'}:  20%|▏| 2/10 [01:33<06:06, 45

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:34<02:18, 34.71s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/st

cross-validation:  40%|██████████▊                | 2/5 [00:28<00:42, 14.13s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:42<00:28, 14.19s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:56<00:14, 14.18s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:10<00:00, 14.19s/it]

{'experiment': '243sentences', 'atlas': 'MD'}: 100%|█| 10/10 [10:51<00:00, 65.13
cartesian product:   0%|                                 | 0/10 [00:00<?, ?it/s]
{'experiment': '384sentences', 'atlas': 'language'}:   0%| | 0/10 [00:00<?, ?it/

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:10<00:41, 10.27s/it]/home/rw

...



cross-validation: 100%|███████████████████████████| 5/5 [02:49<00:00, 33.80s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:20<11:51, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:20<11:51, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:04<00:17,  4.27s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|██████████▊                | 2/5 [00:08<00:12,  4.



cross-validation: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.10s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:50<07:36, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:50<07:36, 50.75

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:08<00:32,  8.05s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:16<00:24,  8.01s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:24<00:16,  8.00s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:32<00:07,  7.99s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:40<00:00,  8.00s/it]

{'experiment': '384sentences', 'atlas': 'DMN'}:  20%|▏| 2/10 [01:31<05:56, 44.58
{'experiment': '384sentences', 'atlas': 'visual'}:  20%|▏| 2/10 [01:31<05:56, 44

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documen

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.76s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:27<00:51, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:27<00:51, 51.01s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:14<00:57, 14.30s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:28<00:42, 14.23s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:42<00:28, 14.14s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:56<00:14, 14.16s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:10<00:00, 14.14s/it]

{'experiment': '243sentences', 'atlas': 'MD'}: 100%|█| 10/10 [10:38<00:00, 63.85
cartesian product:   0%|                                 | 0/10 [00:00<?, ?it/s]
{'experiment': '384sentences', 'atlas': 'language'}:   0%| | 0/10 [00:00<?, ?it/

cross-validation:  

cross-validation:  80%|█████████████████████▌     | 4/5 [01:31<00:23, 23.00s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:54<00:00, 22.98s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:37<07:11, 86.35s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:37<07:11, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:06<00:24,  6.22s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:12<00:18,  6.30s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:18<00:12,  6.29s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:25<00:06,  6.29s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:31<00:00,  6.29s/it]

{'experiment': '243sentences', 'atlas': 'language'}:  60%|▌| 6/10 [07:09<04:31, 
{'experiment': '243sentences', 'atlas': 'DMN'}:  60%|▌| 6/10 [07:09<04:31, 67.76

cross-validation: 

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.03099828, 0.04411379],
       [0.7927616 , 0.1345699 ],
       [0.88272344, 0.15870328],
       [0.87664351, 0.16125136],
       [0.87573072, 0.14125894],
       [0.90044105, 0.12196961],
       [0.88420809, 0.13980054],
       [0.90159703, 0.13986592],
       [0.89928501, 0.14760955],
       [0.87566467, 0.11149804],
       [0.86834772, 0.08572715],
       [0.87622264, 0.13583578],
       [0.86621181, 0.09970486],
       [0.85538056, 0.12562381],
       [0.86774062, 0.12345441],
       [0.8669368 , 0.15156227],
       [0.85444689, 0.14278467],
       [0.85243854, 0.13611955],
       [0.85436388, 0.12010337],
       [0.87313079, 0.10995643],
       [0.84102334, 0.10547166],
       [0.84075276, 0.10676029],
       [0.83865178, 0.10989923],
       [0.84503686, 0.12273717],
       [0.84454688, 0.12515378]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00987...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-1000-embedding-gpt2-test
    benchmark:    Pereira2018-encoding